# Part 2 - Cleaning of Economic Data

### Data Dictionary

|Goal|File Name|Indicator|Description|
|-----|----|---|---|
|Goal 1|t111|1.1.1|Proportion of the population living below the international poverty line by sex, age, employment status and geographic location (urban/rural)|
|Goal 4 |t411|4.1.1|Proportion of children and young people (a) in grades 2/3; (b) at the end of primary; and (c) at the end of lower secondary achieving at least a minimum proficiency level in (i) reading and (ii) mathematics, by sex|
|Goal 14 |t14c1|14.c.1|Number of countries making progress in ratifying, accepting and implementing through legal, policy and institutional frameworks, ocean-related instruments that implement international law, as reflected in the United Nations Convention on the Law of the Sea, for the conservation and sustainable use of the oceans and their resources|
|Goal 9 |t9.5.1|9.5.1|Research and development expenditure as a proportion of GDP|
|Goal 9 |t9.5.2|9.5.2|Researchers (in full-time equivalent) per million inhabitants|

# Libraries

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Data

In [2]:
t111 = pd.read_csv('../sdg_piracy_sheets/csv_data/t111.csv', index_col=[0])
t411 = pd.read_csv('../sdg_piracy_sheets/csv_data/t411.csv', index_col=[0])
t14c1 = pd.read_csv('../sdg_piracy_sheets/csv_data/t14c1.csv', index_col=[0])
rd_gdp = pd.read_csv('../sdg_piracy_sheets/csv_data/t9.5.1.csv', index_col=[0])
research = pd.read_csv('../sdg_piracy_sheets/csv_data/t9.5.2.csv', index_col=[0])

* Decided to drop the following columns for the sake of further clean up

In [3]:
drop_lis = ['Goal', 'Target', 'SeriesCode', 'SeriesDescription', 'FootNote']
t111.drop(drop_lis, axis=1, inplace=True)
t411.drop(drop_lis, axis=1, inplace=True)
t14c1.drop(drop_lis, axis=1, inplace=True)

* Inclusion of UN Poverty data/statistics to the dataset, SDG goal 1.  

In [4]:
poverty= t111[t111['Age']!='15-24']
poverty= poverty[poverty['Age']!= '15+']
poverty= poverty[poverty['Age']!= '25+']
poverty= poverty[poverty['Sex']!= 'MALE']
poverty= poverty[poverty['Age']!= 'FEMALE']

In [5]:
drop_lis = ['Age', 'Sex', 'Time_Detail', 'Source', 'Nature', 'Reporting Type', 'Units']
poverty.drop(drop_lis, axis=1, inplace=True)

In [6]:
poverty.rename(columns={'Value':'poverty_percentage'}, inplace=True)
poverty['GeoAreaCode'] = poverty['GeoAreaCode'].astype(int)

* Imputed missing values found within a particular year with the previous years' measurement.

[source](https://stackoverflow.com/questions/65817843/how-to-create-new-rows-with-missing-years-and-populate-them-with-current-rows)

In [7]:
def year_filler(df, low=2000, high=2022):
    df = (df.set_index('TimePeriod').groupby('GeoAreaCode').apply(lambda x: x.reindex(np.arange(low, high)).ffill()).reset_index('GeoAreaCode',drop=True).reset_index())
    df.dropna(inplace=True)
    return df

* Analyzed observations from 2010 onwards. 

In [8]:
poverty = year_filler(poverty)
poverty['join_key'] = list(zip(poverty['GeoAreaCode'], poverty['TimePeriod']))
poverty = poverty[poverty['TimePeriod'] > 2009]

In [9]:
poverty =poverty.drop(['TimePeriod', 'Indicator', 'GeoAreaCode', 'GeoAreaName'], axis=1)

In [10]:
poverty.to_csv('../datasets/poverty.csv')

* Inclusion of math and reading data/statistics, SDG goal 4.

In [11]:
t411= t411[t411['Sex']!= 'MALE']
t411= t411[t411['Sex']!= 'FEMALE']
t411= t411[t411['Education level']== 'LOWSEC']
t411m = t411[t411['Type of skill']== 'SKILL_MATH']
t411r = t411[t411['Type of skill']== 'SKILL_READ']

In [12]:
t411m = year_filler(t411m)
t411r = year_filler(t411r)

In [13]:
t411m['GeoAreaCode'] = t411m['GeoAreaCode'].astype(int)
t411m['join_key'] = list(zip(t411m['GeoAreaCode'], t411m['TimePeriod']))
math = t411m[['join_key', 'Value']]

t411r['GeoAreaCode'] = t411r['GeoAreaCode'].astype(int)
t411r['join_key'] = list(zip(t411r['GeoAreaCode'], t411r['TimePeriod']))
reading = t411r[['join_key', 'Value']]

math.rename(columns={'Value': 'math_benchmark_percent'}, inplace=True)
reading.rename(columns={'Value': 'reading_benchmark_percent'}, inplace=True)

* Saved to respective csv files after cleaning.

In [14]:
math.to_csv('../datasets/math.csv')
reading.to_csv('../datasets/reading.csv')

* Inclusion of sea law data/statistics, SDG goal 14.

In [15]:
sea_law = t14c1[['GeoAreaCode', 'Value']]
sea_law.rename(columns={'Value': 'sea_law_adoption', 'GeoAreaCode': 'country_code'}, inplace=True)

In [16]:
sea_law.to_csv('../datasets/sea_law.csv')

Inclusion of R&D data/statistics, SDG goal 9.

In [17]:
rd_gdp.head()

,GeoAreaCode,GeoAreaName,TimePeriod,R&D proportion of GDP
Indicator,,,,
9.5.1,8,Albania,2007,0.08757
9.5.1,8,Albania,2008,0.15412
9.5.1,12,Algeria,2001,0.23028
9.5.1,12,Algeria,2002,0.36639
9.5.1,12,Algeria,2003,0.19622


In [18]:
rd_gdp = year_filler(rd_gdp)
rd_gdp = rd_gdp[rd_gdp['TimePeriod'] > 2009]

In [19]:
rd_gdp.rename(columns={'R&D proportion of GDP ':'R&D_proportion_of_GDP'}, inplace=True)

rd_gdp['GeoAreaCode'] = rd_gdp['GeoAreaCode'].astype(int)
rd_gdp['join_key'] = list(zip(rd_gdp['GeoAreaCode'], rd_gdp['TimePeriod']))
rd_gdp = rd_gdp[['join_key', 'R&D_proportion_of_GDP']]

In [20]:
rd_gdp.to_csv('../datasets/rd_gdp.csv')

In [21]:
research.rename(columns={' Full Time Researchers per million (pop)  ':'full_time_researchers_per_million', 'Year': 'TimePeriod'}, inplace=True)
research.head()

,GeoAreaCode,GeoAreaName,TimePeriod,full_time_researchers_per_million,Time_Detail
Indicator,,,,,
9.5.2,8,Albania,2008,155.52783,2008
9.5.2,12,Algeria,2005,168.71935,2005
9.5.2,12,Algeria,2017,819.34270,2017
9.5.2,16,American Samoa,2002,84.63101,2002
9.5.2,16,American Samoa,2003,403.33423,2003


In [22]:
research = year_filler(research)
research = research[research['TimePeriod'] > 2009]

In [23]:
research['GeoAreaCode'] = research['GeoAreaCode'].astype(int)
research['join_key'] = list(zip(research['GeoAreaCode'], research['TimePeriod']))
research = research[['join_key', 'full_time_researchers_per_million']]

In [24]:
research.to_csv('../datasets/research.csv')

* All cleaned SDG data are saved to their respective csv files. 

* Merging of the SDG, economic data above with the our pirate activity data cleaned in part 1 in order to produce a master data frame we will then used for or EDA and modeling steps. 

In [25]:
df = pd.read_csv('../datasets/cleaned_pirate_activity_eda.csv', index_col=[0])
df['join_key'] = list(zip(df.country_code, df.year))

* Counted the number of attacks per country within our scope.

In [26]:
#counting attacks by country
attacks = df.groupby('country_code').size().to_frame()
attacks.rename(columns = {0:'attacks'}, inplace = True)

In [27]:
#joining features from other data
df = df.merge(poverty, on='join_key', how='left')
df = df.merge(attacks, on='country_code', how='left')
df = df.merge(math, on='join_key', how='left')
df = df.merge(reading, on='join_key', how='left')
df = df.merge(sea_law, on='country_code', how='left')
df = df.merge(rd_gdp, on='join_key', how='left')
df = df.merge(research, on='join_key', how='left')

* Sanity check to see if there are any null values after our merge.

In [28]:
for col in df.columns.tolist():
    print(col, df[col].isna().sum())

X 0
Y 0
subreg 0
hostilitytype_l 0
victim_l 0
navarea 0
coords 0
country 0
year 0
month 0
day 0
subreg_11 0
subreg_12 0
subreg_13 0
subreg_18 0
subreg_21 0
subreg_22 0
subreg_23 0
subreg_24 0
subreg_25 0
subreg_26 0
subreg_27 0
subreg_28 0
subreg_36 0
subreg_37 0
subreg_44 0
subreg_51 0
subreg_52 0
subreg_53 0
subreg_54 0
subreg_55 0
subreg_56 0
subreg_57 0
subreg_61 0
subreg_62 0
subreg_63 0
subreg_71 0
subreg_72 0
subreg_73 0
subreg_74 0
subreg_81 0
subreg_82 0
subreg_83 0
subreg_91 0
subreg_92 0
subreg_93 0
subreg_94 0
subreg_95 0
subreg_96 0
subreg_97 0
hostilitytype_l_attempted_boarding 0
hostilitytype_l_hijacking 0
hostilitytype_l_kidnapping 0
hostilitytype_l_navel_engagement 0
hostilitytype_l_other 0
hostilitytype_l_pirate_assaults 0
hostilitytype_l_suspicious_approach 0
hostilitytype_l_unknown 0
victim_l_anchored_vessel 0
victim_l_barge 0
victim_l_cargo_ship 0
victim_l_fishing_vessel 0
victim_l_merchant_vessel 0
victim_l_offshore_vessel 0
victim_l_other 0
victim_l_passenger_shi

In [29]:
df['math_benchmark_percent'].fillna(0,inplace=True)
df['reading_benchmark_percent'].fillna(0,inplace=True)
df['sea_law_adoption'].fillna(0,inplace=True)
df['poverty_percentage'].fillna(25,inplace=True)
df['R&D_proportion_of_GDP'].fillna(25,inplace=True)
df['full_time_researchers_per_million'].fillna(10,inplace=True)

* From our _'high_risk'_ column we deemed countries that were more prone to attacks, more than 15 attacks, as high risk whilst countries below this threshold were deemed low risk countries. 

In [30]:
#creating a y/target for modeling
df['high_risk'] = np.where(df['attacks']<15, 0, 1)

In [31]:
df.to_csv('../datasets/eda_df.csv')

In [32]:
#Removing incidents not near a country
df = df[df['country_code']!=1]

In [33]:
len(df.join_key.unique().tolist())

395

In [34]:
df.drop_duplicates('join_key', inplace=True)

In [35]:
df = df[['year', 'navarea_i','navarea_ii','navarea_iii','navarea_iv','navarea_ix','navarea_v',\
         'navarea_vi','navarea_vii','navarea_viii','navarea_x','navarea_xi','navarea_xii',\
         'navarea_xiii','navarea_xiv','navarea_xv','navarea_xvi','poverty_percentage','attacks',\
         'math_benchmark_percent', 'reading_benchmark_percent','sea_law_adoption','R&D_proportion_of_GDP',\
         'full_time_researchers_per_million','high_risk']]

In [36]:
#finding baseline. I increased the cutoff here and it resulted in better modeling
df.high_risk.value_counts(normalize=True)

1    0.607595
0    0.392405
Name: high_risk, dtype: float64

In [37]:
df.to_csv('../datasets/modeling.csv')

The final merged and cleaned data is saved into it's respective csv file. This is what we will use for our exploratory data analysis (part 3) and modeling (part 4). 